In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Dropout
from keras.callbacks import Callback, ModelCheckpoint
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.optimizers import SGD
from keras.applications.vgg16 import preprocess_input

/Users/wangqiang/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
IMG_WIDTH, IMG_HEIGHT = 64, 64
TRAIN_DATA_DIR = 'input_data'
MODEL_WEIGHTS_FILE = 'vgg16-xfer-tuned-weights.h5'
NB_TRAIN_SAMPLES = 2000
NB_VALIDATION_SAMPLES = 800
NB_EPOCH = 50

In [3]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        TRAIN_DATA_DIR,
        target_size=(IMG_WIDTH, IMG_HEIGHT),
        batch_size=32,
        class_mode='categorical')

Found 3037 images belonging to 4 classes.


In [6]:
model_vgg16_conv = VGG16(weights='imagenet', include_top=False)
for layer in model_vgg16_conv.layers[:-4]:
    layer.trainable = False

In [7]:
input = Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
x = model_vgg16_conv(input)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(4, activation='softmax')(x)
model = Model(input=input, output=x)
model.compile(loss='categorical_crossentropy',
        optimizer=SGD(lr=1e-4, momentum=0.9),
        metrics=['accuracy'])

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/Users/wangqiang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  import sys


In [14]:
callbacks = [ModelCheckpoint(MODEL_WEIGHTS_FILE, monitor='val_acc', save_best_only=True)]
history = model.fit_generator(
        train_generator,
        callbacks = callbacks,
        samples_per_epoch=NB_TRAIN_SAMPLES,
        nb_epoch=NB_EPOCH,
        nb_val_samples=NB_VALIDATION_SAMPLES)

/Users/wangqiang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  import sys
/Users/wangqiang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., callbacks=[<keras.ca..., steps_per_epoch=62, epochs=50, validation_steps=800)`
  import sys


Epoch 1/50
62/62 [==============================] - 99s 2s/step - loss: 0.3953 - acc: 0.8634
Epoch 2/50


/Users/wangqiang/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:434: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


62/62 [==============================] - 98s 2s/step - loss: 0.3570 - acc: 0.8723
Epoch 3/50
62/62 [==============================] - 61s 987ms/step - loss: 0.3796 - acc: 0.8594
Epoch 4/50
62/62 [==============================] - 61s 984ms/step - loss: 0.3247 - acc: 0.8873
Epoch 5/50
62/62 [==============================] - 61s 990ms/step - loss: 0.3617 - acc: 0.8728
Epoch 6/50
62/62 [==============================] - 62s 1s/step - loss: 0.3620 - acc: 0.8785
Epoch 7/50
62/62 [==============================] - 61s 984ms/step - loss: 0.3559 - acc: 0.8672
Epoch 8/50
62/62 [==============================] - 59s 955ms/step - loss: 0.3564 - acc: 0.8755
Epoch 9/50
62/62 [==============================] - 62s 1s/step - loss: 0.3395 - acc: 0.8814
Epoch 10/50
62/62 [==============================] - 85s 1s/step - loss: 0.3550 - acc: 0.8650
Epoch 11/50
62/62 [==============================] - 61s 990ms/step - loss: 0.3355 - acc: 0.8866
Epoch 12/50
62/62 [==============================] - 61s 985m

KeyboardInterrupt: 

In [15]:
model.save('model.h5')